**MODELOS DE EMBEDDINGS BASADOS EN WORD2VEC**

*John Atkinson*

Este programa utiliza  crea y utiliza modelos de embeddings de lenguaje basado en mètodos del tipo Word2Vec.

Primero, necesitamos instalar algunos paquetes:

In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm
%pip install gensim

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Note: you may need to restart the kernel to use updated packages.


Montamos nuestro *Drive* donde se encuentra la carpeta CORPUS con documentos separados por tema:

Debemos importar algunas bibliotecas  y utilitarios:

In [2]:
import es_core_news_sm
from string import punctuation
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import regex
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import os
import matplotlib.pyplot as plt
import glob
import unicodedata
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/CORPUS

In [3]:

file_paths = glob.glob("corpus/*")
corpus = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        content = file.read()
        corpus.append(content)


Primero, definimos la función **EntrenarModelo(oraciones,NombreModelo)**, que permite entrenar un modelo Word2Vec a partir de un conjunto de oraciones extraída desde un corpus. El modelo generado se graba en la carpeta **NombreModelo**. Asumimos que la ventaja de entrenamiento es 2 ($windows=2$) y que el número de dimensiones o tamaño del vector es 8 ($size=8$). Note que se debe encontrar el mejor tamaño del embedding.

Luego, definimos una función **CargarModelo(NombreModelo)**, que nos permitirá cargar un modelo cuando sea necesario.

Note que no necesariamente debemos entrenar el modelo nosotros mismos. Podríamos cargar un modelo de embeddings que ya ha sido entrenado por alguien más.

In [4]:
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones, vector_size=300, window=2, min_count=1)
    model.save(NombreModelo)

def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.key_to_index]
   return(modelo,vocabulario)

Además, necesitaremos una función **ObtenerEmbeddingOracion(modelo, oracion)**, que nos permitirá obtener el embedding (vector) de una oración desde un modelo entrenado. El embedding de una oración es simplemente el vector promedio de cada una de las palabras de la oración:

In [5]:
def ObtenerEmbeddingOracion(modelo, oracion):
   Lista_vectores = []
   for w in Tokenizar(oracion):
       # Verificar que la palabra w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la palabra
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_palabras = np.array(Lista_vectores)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(modelo.vector_size)
   return(embedding_oracion)


Ahora, utilizamos algunas funciones de preprocesamiento:
(lematizar, eliminar stopwords y tokenizar)

In [6]:
def PreProcesarOraciones(textos):
    texto_limpio = []
    for texto in textos:
        if len(texto)!=0:
          texto = regex.sub(' +', ' ', texto)
          tokens = Tokenizar(texto)
          texto_limpio.append(tokens)
    return(texto_limpio)

def Tokenizar(oracion):
    doc = nlp(oracion)
    tokens = [palabra.text for palabra in doc]
    return(tokens)

def ProcesarTexto(oracion):
    doc = nlp(oracion)
    tokens = [palabra.lemma_ for palabra in doc if not palabra.is_stop]
    return tokens

def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="latin-1").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus, doc_id)

En caso de ser necesario definimos una función que permite convertir una lista a un diccionario, de modo de poder acceder por clave (y no por índice):

In [7]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

Ahora, ejecutamos nuestro programa principal con algunas incializaciones:

In [8]:
PATH = "corpus/"
nlp          = es_core_news_sm.load()
corpus, docID = CrearCorpus(PATH)
oraciones    =  PreProcesarOraciones(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)

Entrenamos el modelo en base a las oraciones generadas previamente:

In [9]:
EntrenarModelo(oraciones,'mi_word2vec')

Luego, cargamos el modelo entrenado:

In [10]:
modelo, vocabulario = CargarModelo('mi_word2vec')

Podemos, obtener el embedding de alguna palabra:

Una vez que tenemos nuestro modelo cargado, podemos realizar diferentes tareas sobre los vectores de palabras u oraciones.

Por ejemplo, podemos determinar la *cercanía* entre dos documentos del corpus. Para ello:

1.   Tomamos el texto de cada documento.
2.   Obtenemos sus respectivos vectores (embeddings).
3.   Calculamos la distancia **coseno**.

Note que la calidad del resultado depende de la cercanía real de los documentos!

Solicitamos al Usuario las palabras para comparar y mostramos las primeras 30 similitudes

In [13]:

query = input("Ingresa tus palabras para comparar: ")


query_embedding = ObtenerEmbeddingOracion(modelo, query)


similitud_puntajes = []
for documentos in corpus:
    documentos_embedding = ObtenerEmbeddingOracion(modelo, documentos)
    similitud = 1 - cosine(query_embedding, documentos_embedding)
    similitud_puntajes.append(similitud)


top_indices = np.argsort(similitud_puntajes)[::-1][:30]


for index in top_indices:
    documentos_name = docID[index]
    similitud = similitud_puntajes[index]
    print(f"Documento: {documentos_name}, Similitud: {similitud}")


Documento: 149691.txt, Similitud: 0.6409827470779419
Documento: 149471.txt, Similitud: 0.625017523765564
Documento: 150206.txt, Similitud: 0.6183123588562012
Documento: 97464.txt, Similitud: 0.6145167946815491
Documento: 152647.txt, Similitud: 0.6103097200393677
Documento: 135384.txt, Similitud: 0.6055464744567871
Documento: 84418.txt, Similitud: 0.6054936051368713
Documento: 151662.txt, Similitud: 0.6044436693191528
Documento: 82199.txt, Similitud: 0.6034389138221741
Documento: 85692.txt, Similitud: 0.6033058762550354
Documento: 103681.txt, Similitud: 0.6029363870620728
Documento: 82440.txt, Similitud: 0.6006869673728943
Documento: 103651.txt, Similitud: 0.6001328825950623
Documento: 74127.txt, Similitud: 0.5995113253593445
Documento: 134150.txt, Similitud: 0.5983396172523499
Documento: 152727.txt, Similitud: 0.5983177423477173
Documento: 151583.txt, Similitud: 0.5976815819740295
Documento: 150645.txt, Similitud: 0.5976508855819702
Documento: 151937.txt, Similitud: 0.597638726234436
D